In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

spark= SparkSession.builder.appName('task_1').getOrCreate()

data=[
    ('A',1,'5/23/2003',100),
    ('B',2,'5/23/2003',200),
    ('A',4,'5/23/2003',200),
    ('B',3,'5/22/2003',200),
    ('A',5,'5/21/2003',500),
    ('A',1,'5/20/2003',600),
    ('C',2,'5/23/2003',400),
    ('A',4,'5/23/2003',200),
    ('C',6,'5/22/2003',400),
    ('A',5,'5/21/2003',500),
]
df= spark.createDataFrame(data,['GRADE','ROll_NO','DATE','MARKS'])
df = df.withColumn("DATE", to_date("DATE", "M/d/yyyy"))
# sorted_df2 = df.orderBy("Date")

display(df)

GRADE,ROll_NO,DATE,MARKS
A,1,2003-05-23,100
B,2,2003-05-23,200
A,4,2003-05-23,200
B,3,2003-05-22,200
A,5,2003-05-21,500
A,1,2003-05-20,600
C,2,2003-05-23,400
A,4,2003-05-23,200
C,6,2003-05-22,400
A,5,2003-05-21,500


In [0]:

def transform_dataframe(input_df):

    dfs = []
    for i in input_df.columns:
 
        id_count = input_df.groupBy(i).agg(count('*').alias('count'))
        id_count_sorted = id_count.orderBy(['count', i], ascending=[False, True])
        percentage = format_number((id_count_sorted['count'] / input_df.count()) * 100, 1)
       
        percentage_with_percent = concat(i,lit(' - '),percentage, lit('%'))
      
        value_count_df = id_count_sorted.withColumn(i, percentage_with_percent)
       
        value_count_df = value_count_df.select(i)
        
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
     
        dfs.append(value_count_df)
      
        x = dfs[0]
        for i in dfs[1:]:
            x = x.join(i, "ID", "full")


    return x.drop("ID")

transformed_df = transform_dataframe(df)

display(transformed_df)


GRADE,ROll_NO,DATE,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
B - 20.0%,2 - 20.0%,2003-05-21 - 20.0%,400 - 20.0%
C - 20.0%,4 - 20.0%,2003-05-22 - 20.0%,500 - 20.0%
null,5 - 20.0%,2003-05-20 - 10.0%,100 - 10.0%
null,3 - 10.0%,null,600 - 10.0%
null,6 - 10.0%,null,null


In [0]:
from pyspark.sql.functions import col, concat, lit

def transform_dataframe(input_df):
    dfs = []
    for i in input_df.columns:
        id_count = input_df.groupBy(i).agg(count('*').alias('count'))
        total_count = input_df.count()
        id_count_with_percentage = id_count.withColumn("percentage", (id_count['count'] / total_count) * 100)
        max_percentage = id_count_with_percentage.agg(spark_max("percentage")).collect()[0][0]
        max_percentage_rows = id_count_with_percentage.filter(id_count_with_percentage["percentage"] == max_percentage)
        value_count_df = max_percentage_rows.withColumn(i, concat(i, lit(' - '), col("percentage").cast("string"), lit('%')))
        value_count_df = value_count_df.select(i)
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
        dfs.append(value_count_df)
      
    x = dfs[0]
    for i in dfs[1:]:
        x = x.join(i, "ID", "full")

    return x.drop("ID")

transformed_df = transform_dataframe(df)
display(transformed_df)


GRADE,ROll_NO,DATE,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
null,2 - 20.0%,null,null
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null


In [0]:
from pyspark.sql.functions import col, concat, lit

def transform_dataframe(input_df):
    dfs = []
    for i in input_df.columns:
        id_count = input_df.groupBy(i).agg(count('*').alias('count'))
        total_count = input_df.count()
        id_count_with_percentage = id_count.withColumn("percentage", (id_count['count'] / total_count) * 100)
        sorted_id_count = id_count_with_percentage.orderBy(col("percentage").desc())
        max_percentage = sorted_id_count.select("percentage").first()[0]
        second_max_percentage = sorted_id_count.select("percentage").limit(2).collect()[-1][0]
        rows_with_max_percentage = id_count_with_percentage.filter(col("percentage") == max_percentage)
        rows_with_second_max_percentage = id_count_with_percentage.filter(col("percentage") == second_max_percentage)
        combined_rows = rows_with_max_percentage.union(rows_with_second_max_percentage)
        value_count_df = combined_rows.withColumn(i, concat(i, lit(' - '), col("percentage").cast("string"), lit('%')))
        value_count_df = value_count_df.select(i)
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
        dfs.append(value_count_df)
      
    x = dfs[0]
    for i in dfs[1:]:
        x = x.join(i, "ID", "full")

    return x.drop("ID")

transformed_df = transform_dataframe(df)
display(transformed_df)


GRADE,ROll_NO,DATE,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
null,2 - 20.0%,null,null
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null
B - 20.0%,1 - 20.0%,2003-05-21 - 20.0%,500 - 20.0%
C - 20.0%,2 - 20.0%,2003-05-22 - 20.0%,400 - 20.0%
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null


In [0]:
from pyspark.sql.functions import col, concat, lit, monotonically_increasing_id

def transform_dataframe(input_df):
    dfs = []
    for i in input_df.columns:
        id_count = input_df.groupBy(i).count()
        total_count = input_df.count()
        id_count_with_percentage = id_count.withColumn("percentage", (id_count['count'] / total_count) * 100)
        sorted_id_count = id_count_with_percentage.orderBy(col("percentage").desc())
        max_percentage = sorted_id_count.select("percentage").first()[0]
        second_max_percentage = sorted_id_count.select("percentage").limit(1).collect()[-1][0]
        rows_with_max_percentage = id_count_with_percentage.filter(col("percentage") == max_percentage)
        rows_with_second_max_percentage = id_count_with_percentage.filter(col("percentage") == second_max_percentage)
        combined_rows = rows_with_max_percentage.union(rows_with_second_max_percentage)
        # Concatenate the percentage to each column value
        value_count_df = combined_rows.withColumn(i, concat(i, lit(' - '), col("percentage").cast("string"), lit('%')))
        # Select only the transformed column
        value_count_df = value_count_df.select(i)
        # Add a unique identifier column for joining
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
        dfs.append(value_count_df)
      
    # Join the DataFrames column-wise
    x = dfs[0]
    for i in range(1, len(dfs)):
        x = x.join(dfs[i], "ID", "full")

    # Sort the DataFrame to place null values at the end
    sorted_columns = [col for col in x.columns if col != "ID"]
    x = x.select(sorted_columns)
    
    return x

transformed_df = transform_dataframe(df)
display(transformed_df)


GRADE,ROll_NO,DATE,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
null,2 - 20.0%,null,null
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
null,2 - 20.0%,null,null
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null


In [0]:
from pyspark.sql.functions import col, concat, lit, monotonically_increasing_id

def transform_dataframe(input_df, N):
    dfs = []
    for i in input_df.columns:
        id_count = input_df.groupBy(i).count()
        total_count = input_df.count()
        id_count_with_percentage = id_count.withColumn("percentage", (id_count['count'] / total_count) * 100)
        sorted_id_count = id_count_with_percentage.orderBy(col("percentage").desc())
        max_percentage = sorted_id_count.select("percentage").limit(N).collect()[-1][0]
        rows_with_max_percentage = id_count_with_percentage.filter(col("percentage") == max_percentage)
        value_count_df = rows_with_max_percentage.withColumn(i, concat(i, lit(' - '), col("percentage").cast("string"), lit('%')))
        value_count_df = value_count_df.select(i)
        value_count_df = value_count_df.withColumn("ID", monotonically_increasing_id())
        dfs.append(value_count_df)
      
    x = dfs[0]
    for i in range(1, len(dfs)):
        x = x.join(dfs[i], "ID", "full")

    return x.drop("ID")

# Example usage: Specify the Nth highest percentage you want (e.g., N=1 for the highest percentage)
transformed_df = transform_dataframe(df, N=1)
display(transformed_df)


GRADE,ROll_NO,DATE,MARKS
A - 60.0%,1 - 20.0%,2003-05-23 - 50.0%,200 - 40.0%
null,2 - 20.0%,null,null
null,4 - 20.0%,null,null
null,5 - 20.0%,null,null
